# COMP 352 - Final Project: Footballer Market Evaluation

**Author:** Zevin Attisha, Santi Guerrero, Santiago Pedetti, Christian Vaikona

#### Full dataset: https://www.kaggle.com/datasets/davidcariboo/player-scores/data?select=appearances.csv

## Table of Contents:
* [Environment Setup](#env-setup)
* [Data Importing and Pre-processing](#data-importing)
* [Data Analysis and Visualization](#data-vis)
* [Data Analytics](#data-analytics)

## Environment Setup <a class="anchor" id="env-setup"></a>

(COPIED AND PASTED - EDIT LATER)

First we must setup our environment to make sure we have all appropriate modules installed. To do this, I have provided 2 methods. The 1st, is to install all modules using a ```.yaml``` file via ```conda```. 

To do this, run:
```bash
conda env create -f env_setup/data_environment.yml
```
Then activate the environment by:
```bash
conda activate data_env
```

(COPIED AND PASTED - EDIT LATER)

You can also use the ```requirements.txt``` file to download the modules via ```pip```.

To do this, first make create and activate your environment:
```bash
conda create -n my_data_env
conda activate my_data_env
```

You may need to install setup tools. To do this run (Note you may need to change ```pip3``` to ```pip```):
```bash
pip3 install --upgrade pip setuptools wheel
```

and then run:
```bash
pip3 install -r env_setup/requirements.txt
```

## Data Importing and Pre-processing <a class="anchor" id="data-importing"></a>

Section Overview:

- Import dataset and describe characteristics such as dimensions, data types, file types, and import methods used
- Clean, wrangle, and handle missing data, duplicate data, etc.
- Encode any categorical variables
- Perform feature engineering on the dataset
- Transform data appropriately using techniques such as aggregation, normalization, and feature construction
- Reduce redundant data and perform need based discretization

In [ ]:
# import libraries needed
import pandas as pd

pd.set_option("display.max_columns", None)
import warnings

import branca
import folium
import geopandas as gpd
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import xgboost as xgb
from branca.element import Figure
from folium import Marker
from folium.plugins import HeatMap
from scipy.special import boxcox1p
from scipy.stats import norm, probplot, skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor

from utils.model_scripts import (
    hello
)
from utils.metric_scripts import (
    hello
)

warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FutureWarning, module="pandas.*")
%matplotlib inline

In [15]:
# read in files
player_valuations_df = pd.read_csv("data/raw/player_valuations.csv")
players_df = pd.read_csv("data/raw/players.csv")

In [18]:
# dimensions of each df
print("player_valuations.csv dimensions: " + str(player_valuations_df.shape))
print("players.csv dimensions: " + str(players_df.shape))

player_valuations.csv dimensions: (496606, 5)
players.csv dimensions: (32601, 23)


In [25]:
# count the number of categorical variables

pv_cat_count = 0
for dtype in player_valuations_df.dtypes:
    if dtype == "object":
        pv_cat_count +=1

pv_numeric_vars = player_valuations_df.shape[1] - pv_cat_count - 1 # subtract an extra column as 1 column is an ID column

print("For player_valuations.csv Data:")
print("# of categorical values: " + str(pv_cat_count))
print("# of continuous variables:", str(pv_numeric_vars)+"\n")

p_cat_count = 0
for dtype in players_df.dtypes:
    if dtype == "object":
        p_cat_count +=1

p_numeric_vars = players_df.shape[1] - p_cat_count - 1 # subtract an extra column as 1 column is an ID column

print("For players.csv Data:")
print("# of categorical values: " + str(p_cat_count))
print("# of continuous variables:", str(p_numeric_vars)) 

For player_valuations.csv Data:
# of categorical values: 2
# of continuous variables: 2

For players.csv Data:
# of categorical values: 17
# of continuous variables: 5


In [27]:
player_valuations_df.head()

,player_id,date,market_value_in_eur,current_club_id,player_club_domestic_competition_id
0,405973,2000-01-20,150000,3057,BE1
1,342216,2001-07-20,100000,1241,SC1
2,3132,2003-12-09,400000,126,TR1
3,6893,2003-12-15,900000,984,GB1
4,10,2004-10-04,7000000,398,IT1


In [28]:
players_df.head()

,player_id,first_name,last_name,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,date_of_birth,sub_position,position,foot,height_in_cm,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
0,10,Miroslav,Klose,Miroslav Klose,2015,398,miroslav-klose,Poland,Opole,Germany,1978-06-09 00:00:00,Centre-Forward,Attack,right,184.0,NaN,ASBW Sport Marketing,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,IT1,Società Sportiva Lazio S.p.A.,1000000.0,30000000.0
1,26,Roman,Weidenfeller,Roman Weidenfeller,2017,16,roman-weidenfeller,Germany,Diez,Germany,1980-08-06 00:00:00,Goalkeeper,Goalkeeper,left,190.0,NaN,Neubauer 13 GmbH,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/roman-weidenfe...,L1,Borussia Dortmund,750000.0,8000000.0
2,65,Dimitar,Berbatov,Dimitar Berbatov,2015,1091,dimitar-berbatov,Bulgaria,Blagoevgrad,Bulgaria,1981-01-30 00:00:00,Centre-Forward,Attack,NaN,NaN,NaN,CSKA-AS-23 Ltd.,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/dimitar-berbat...,GR1,Panthessalonikios Athlitikos Omilos Konstantin...,1000000.0,34500000.0
3,77,NaN,Lúcio,Lúcio,2012,506,lucio,Brazil,Brasília,Brazil,1978-05-08 00:00:00,Centre-Back,Defender,NaN,NaN,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/lucio/profil/s...,IT1,Juventus Football Club,200000.0,24500000.0
4,80,Tom,Starke,Tom Starke,2017,27,tom-starke,East Germany (GDR),Freital,Germany,1981-03-18 00:00:00,Goalkeeper,Goalkeeper,right,194.0,NaN,IFM,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/tom-starke/pro...,L1,FC Bayern München,100000.0,3000000.0


## Data Analysis and Visualization <a class="anchor" id="data-vis"></a>

Section Overview:

- Identify categorical, ordinal, and numerical variables within data
- Provide measures of centrality and distribution with visualizations
- Diagnose for correlations between variables and determine independent and dependent variables
- Perform exploratory analysis in combination with visualization techniques to discover patterns and features of interest
- Create visualizations that allow for the discovery of insights in the data

## Data Analytics <a class="anchor" id="data-analytics"></a>